<a id='top'></a>
# Adversarial examples on images generated by 2D Cellular Automaton
----
## Table of Content
----

<a href='#section1'>1. Introduction</a>

<a href='#section2'>2. Cellular Automaton</a>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href='#section2.1'>2.1. Cellular Automaton engine</a>

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<a href='#section2.2'>2.2. Evolution rules</a>

<a href='#section3'>3. Image generation</a>

----

<a id='section1'></a>
## 1. Introduction
<p style="text-align:right;"><a href='#top'>Beginning</a></p>

Cellular automata are mathematical models for systems in which many simple components act together to produce complicated patters of behavior.

A cellular automaton consists of a regular lattice of sites. Each site takes on $k$ possible values, and is updated in discrete time steps according to a rule $\phi$ that depends on the values of sites in some neighborhood around it.

There are several possible lattices and neighborhood structures for two-dimensional cellular automata. For example, five-neighbor (von Neumann) square cellular automaton then evolves according to:

$$a_{i,j}^{(t+1)}=\phi[a_{i,j}^{(t)}, a_{i,j+1}^{(t)}, a_{i+1,j}^{(t)}, a_{i,j-1}^{(t)}, a_{i-1,j}^{(t)}]$$

| ![von Neumann Neighborhood](res/vonNeumann.png) | ![Moore Neighborhood](res/Moore.png) |
| :--: | :--: |
| *von Neumann neighborhood* | *Moore neighborhood* |

__Source__: [Packard, N.H., Wolfram, S. Two-dimensional cellular automata. J Stat Phys 38, 901–946 (1985).](https://content.wolfram.com/uploads/sites/34/2020/07/two-dimensional-cellular-automata.pdf)


In [253]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm

plt.rcParams["animation.html"] = "html5"
plt.rcParams['figure.dpi'] = 150  
plt.ioff()

<a id='section2' style='display: none;'></a>
## 2. Cellular Automaton
<p style="text-align:right;"><a href='#top'>Beginning</a></p>

<a id='section2.1'></a>
### 2.1. Cellular Automaton engine
<p style="text-align:right;"><a href='#top'>Beginning</a></p>

In [297]:
class BinaryCellularAutomaton:
    
    '''
    Class representing the Cellular Automaton with k = 2 (binary).
    
      - Rows (int): represents the number of rows of the cellular automata.
      - Cols (int): represents the number of columns of the cellular automata.
      - evolution_rule (callable): rule to apply each timestep by the cellular automata.
                                   Takes as input a 9x9 matrix, and returns a single value.
      - neighborhood (str): Either 'von Neumann' or 'Moore'.
      - seed (ndarray, optional): matrix of shape (timesteps + 1, rows, cols) indicating the inital 
                                  state of the automaton at slice [0, :, :]. If not passed, the seed is random.
      - timesteps (int, optional): represents the total number of steps to evolve the CA.
    '''
    
    def __init__(self, rows, cols, evolution_rule, neighborhood, seed=None, timesteps=50):
        if neighborhood not in ['von Neumann', 'Moore']:
            raise Exception('Invalid neighborhood type. Valid types are "von Neumann" and "Moore".')
            
        if not callable(evolution_rule):
            raise Exception('Invalid evolution_rule. It should be a callable function.')
        
        seed = None if seed is None else np.copy(seed)
        self.ca = np.random.randint(2, size=(timesteps, rows, cols), dtype=np.bool) if seed is None or seed.shape[1:] != (rows, cols) else seed
        self.evolution_rule = evolution_rule
        self.neighborhood = neighborhood
        self.timesteps = timesteps
    
    # This is a Null-boundary neighborhood
    def _get_neighborhood(self, row, col, timestep=-1):        
        neighborhood = np.zeros((3,3), dtype=np.bool)
        
        for i in range(row - 1, row + 2):
            if i >= 0 and i < self.ca.shape[1]:
                for j in range(col - 1, col + 2):
                    if j >= 0 and j < self.ca.shape[2]:
                        neighborhood[i - row + 1, j - col + 1] = self.ca[timestep, i, j]
                        
        von_neumann_mask = [[False, True, False], [True, True, True], [False, True, False]]
        
        return neighborhood if self.neighborhood == 'Moore' else neighborhood * von_neumann_mask
    
    def evolve(self):
        for t in tqdm(range(self.timesteps - 1)):
            for i in range(self.ca.shape[1]):
                for j in range(self.ca.shape[2]):
                    self.ca[t+1, i, j] = self.evolution_rule(self._get_neighborhood(i,j,timestep=t))
        
    def animate(self):
        fig = plt.figure(figsize=(3,3))
        ax = plt.axes()
        i = 0
        im = ax.imshow(np.abs(1 - self.ca[i, :, :]), cmap=plt.cm.gray, vmin=0, vmax=1, animated=True)
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        
        # Code taken from: https://stackoverflow.com/questions/50413680/matplotlib-animate-2d-array
        def updatefig(*args):
            nonlocal i, self
            if (i<self.timesteps - 1):
                i += 1
            else:
                i=0
            im.set_array(np.abs(1 - self.ca[i]))
            return im,
        
        anim = animation.FuncAnimation(fig, updatefig,  blit=True)
        plt.close()
        
        return anim
    
    def draw(self, timestep=-1):
        plt.figure(figsize=(3,3))
        ax = plt.axes()
        ax.imshow(np.abs(1 - self.ca[timestep - 1, :, :]), cmap=plt.cm.gray, vmin=0, vmax=1)
        plt.suptitle(f'Timestep {timestep if timestep != -1 else self.timesteps}', fontsize=8)
        plt.setp(ax.get_xticklabels(), visible=False)
        plt.setp(ax.get_yticklabels(), visible=False)
        ax.tick_params(axis='both', which='both', length=0)
        plt.show()
        plt.close()
    
    def savefig(self, filename, timestep=-1):
        plt.imsave(filename, self.ca[timestep, :, :], cmap=plt.cm.gray)
        

<a id='section2.2'></a>
### 2.2. Evolution rules
<p style="text-align:right;"><a href='#top'>Beginning</a></p>

In [317]:
class EvolutionRules:
    
    @staticmethod
    def game_of_life(neighborhood):
        alive_neighbors = np.sum(neighborhood) - neighborhood[1,1]
        if neighborhood[1,1]:
            return False if alive_neighbors < 2 or alive_neighbors > 3 else True
        elif alive_neighbors == 3:
            return True
    
    @staticmethod
    def high_life(neighborhood):
        alive_neighbors = np.sum(neighborhood) - neighborhood[1,1]
        if neighborhood[1,1]:
            return False if alive_neighbors < 2 or alive_neighbors > 3 else True
        elif alive_neighbors == 3 or alive_neighbors == 6:
            return True
    
    @staticmethod
    def get_rule(idx):
        if idx > 0 and idx <= 512:
            # TODO: implement this logic (https://arxiv.org/pdf/0804.2346.pdf)
            pass
        else:
            raise ValueError("Rule number out of range.")

### 2.3. Testing Automatas

#### 2.3.1. Conway's Game of Life

In [314]:
ca = BinaryCellularAutomaton(50, 50, EvolutionRules.game_of_life, 'Moore')
ca.evolve()

100%|██████████| 49/49 [00:03<00:00, 14.05it/s]


In [315]:
anim = ca.animate()

In [316]:
anim

<a id='section3'></a>
## 3. Image Generation
<p style="text-align:right;"><a href='#top'>Beginning</a></p>